In [ ]:
import numpy as np
from keras.models import Sequential

In [ ]:
import os

path = "./initial training set"
biteLabels = os.listdir(path)



In [ ]:
biteLabels

In [ ]:
allBiteImageNames = []
allBiteImageLabels = []
from functools import reduce
from operator import concat
from __future__ import print_function  # Py 2.6+; In Py 3k not needed
import random

for i in range (len (biteLabels)):
    bitePath = path + "/" + biteLabels[i]
    #print(bitePath)
    bitePathListDir = os.listdir(bitePath)
    print(len(bitePathListDir))
    biteImageLabels = []
    
    for j in range(len(bitePathListDir)):
        allBiteImageLabels.append(i)
        
    allBiteImageNames += bitePathListDir
    

print(len(allBiteImageNames))
print(len(allBiteImageLabels))

imageNameToLabel = dict(zip(allBiteImageNames,allBiteImageLabels))



print(*imageNameToLabel.items(), sep='\n')

In [ ]:
from sklearn.model_selection import train_test_split

trainImageNames, testImageNames  = train_test_split(allBiteImageNames, test_size=0.2, shuffle=False)    

In [ ]:
print(len(trainImageNames))
print(len(testImageNames))

In [ ]:
partition = {
    'train':trainImageNames,
    'validation':testImageNames
}

In [ ]:
imageNameToLabel

In [ ]:
print(len(partition['train']), len(partition['validation']))

In [ ]:
import numpy as np
import keras
from PIL import Image
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        #self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        #X, y = self.__data_generation(list_IDs_temp)
        X = self.__data_generation(list_IDs_temp)
        return X #, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        #y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = Image.open('data/' + ID).resize((256,256))

            # Store class
            #y[i] = self.labels[ID]

        return X #, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
IMAGE_SIZE = 256
params = {'dim': (4,IMAGE_SIZE, IMAGE_SIZE),
          'batch_size': 8,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}

training_generator = DataGenerator(partition['train'], imageNameToLabel, **params)
validation_generator = DataGenerator(partition['validation'], imageNameToLabel, **params)

In [ ]:
import keras
from tensorflow.keras import models, layers
IMAGE_SIZE = 256
BATCH_SIZE = 8
CHANNELS = 3
EPOCHS = 50
Input_shape=(BATCH_SIZE, IMAGE_SIZE,IMAGE_SIZE, CHANNELS)
n_classes = 2
model = models.Sequential()
layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE,IMAGE_SIZE,1)),
layers.Conv2D(64, kernel_size = (3,3), activation='relu'),   
layers.MaxPooling2D((2,2)),
layers.Conv2D(64, kernel_size = (3,3), activation='relu'),              
layers.Conv2D(64, (3,3), activation='relu'),              
layers.MaxPooling2D((2,2)),
layers.Flatten(),
layers.Dense(64, activation='relu'),
layers.Dense(n_classes, activation='sigmoid')
model.build(input_shape=Input_shape)

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
model.compile(
    optimizer ='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']
)

In [ ]:
model.fit(training_generator,
                    validation_generator,
                    use_multiprocessing=True,
                    workers=6)